In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib
import skvideo.io
import gc
import h5py
from collections import defaultdict
import cv2
from scipy.ndimage import gaussian_filter1d

# from utils import *

In [3]:
features = {}

In [4]:
# read DLC tracking
df = pd.read_hdf('A1_bodyDLC_resnet50_palmreader-500Mar25shuffle1_500000.h5')
label = df['DLC_resnet50_palmreader-500Mar25shuffle1_500000']

label

bodyparts     tailtip                           tailbase              \
coords              x           y likelihood           x           y   
0          387.548645   55.379059        1.0  440.261536  242.860077   
1          390.220917   52.890076        1.0  440.291077  242.722916   
2          389.508667   51.362377        1.0  440.822327  241.677277   
3          390.940125   51.681019        1.0  441.844238  241.178238   
4          390.440338   52.041336        1.0  440.397186  242.198868   
...               ...         ...        ...         ...         ...   
44995      306.757538  287.272217        1.0  144.625137  186.873962   
44996      306.333282  287.958801        1.0  142.909225  186.986877   
44997      304.161041  289.132080        1.0  137.716660  185.676361   
44998      302.185883  290.322235        1.0  135.609985  185.994812   
44999      298.991913  291.522186        1.0  133.125900  186.181381   

bodyparts               centroid                               neck  ...  \
coords    likelihood           x           y likelihood           x  ...   
0                1.0  440.001190  341.249420   1.000000  442.685974  ...   
1                1.0  441.671143  339.647705   1.000000  442.674866  ...   
2                1.0  442.995148  340.083344   1.000000  442.606720  ...   
3                1.0  445.291870  339.463165   1.000000  442.536469  ...   
4                1.0  445.850494  339.751923   1.000000  441.074310  ...   
...              ...         ...         ...        ...         ...  ...   
44995            1.0  116.397598  102.043030   0.999991  145.853165  ...   
44996            1.0  116.056305  102.717201   0.999992  149.663666  ...   
44997            1.0  115.198349  102.692810   0.999997  151.402298  ...   
44998            1.0  114.711845  101.192635   0.999998  153.792694  ...   
44999            1.0  115.006065   99.602074   0.999997  156.526321  ...   

bodyparts      hlpaw       hrpaw                              flpaw  \
coords    likelihood           x           y likelihood           x   
0           1.000000  486.943970  268.303101        1.0  384.644165   
1           1.000000  487.033264  268.369507        1.0  384.569916   
2           1.000000  486.728607  268.284424        1.0  384.944702   
3           1.000000  486.826874  268.467834        1.0  405.966187   
4           1.000000  486.861542  268.612732        1.0  429.695953   
...              ...         ...         ...        ...         ...   
44995       0.999989   85.707298  129.020950        1.0  142.625427   
44996       0.999987   85.413162  128.904282        1.0  142.662521   
44997       0.999988   85.412598  129.189224        1.0  142.501587   
44998       0.999985   85.251877  129.173065        1.0  143.745316   
44999       0.999983   85.173691  129.487839        1.0  143.535156   

bodyparts                              frpaw                         
coords              y likelihood           x           y likelihood  
0          378.748138   0.999997  445.851990  412.121674   0.999965  
1          380.100922   0.999994  445.913391  411.535828   0.999939  
2          382.979584   0.999993  445.821869  410.749481   0.999941  
3          415.470398   0.999994  445.330017  412.327362   0.999954  
4          422.875885   0.999988  443.793182  417.523468   0.999912  
...               ...        ...         ...         ...        ...  
44995       61.971542   0.999878  112.346703   54.582657   0.999969  
44996       61.100266   0.999848  117.535500   53.090343   0.999952  
44997       60.660606   0.999971  122.314049   52.751007   0.999976  
44998       60.780117   0.999995  129.638748   51.384567   0.999998  
44999       60.966640   0.999973  134.128326   50.532227   0.999994  

[45000 rows x 27 columns]

In [5]:
# calculate distance traveled
def cal_distance_(label, bodypart = 'centroid'):
    '''helper function for "calculate distance traveled'''
    x = gaussian_filter1d(label[bodypart]['x'].values, 3)
    y = gaussian_filter1d(label[bodypart]['y'].values, 3)
    d_x = np.diff(x)
    d_y = np.diff(y)
    d_location = np.sqrt(d_x**2 + d_y**2)
    return d_location

features['distance_traveled'] = np.nansum(cal_distance_(label)).reshape(-1,1)

In [6]:
# center and align body_pose video

def four_point_transform(image, tx,ty, cx,cy, wid, length):
    '''
    helper function for center and align a single video frame
    input:
        T, coord of tailbase, which is used to center the mouse
        TN, vector from tailbase to centroid
        wid, the width of the to-be-cropped portion
        length, the length of the to-be-cropped portion
        
    output:
        warped: the cropped portion in the size of (wid, length), mouse will be centered by tailbase, aligned by the direction from tailbase to centroid

    '''
    T = np.array([tx,ty])
    N = np.array([cx,cy])
    TN = N - T
    
    uTN = TN / np.linalg.norm(TN) # calculate the unit vector for TN
    
    # calculate the unit vector perpendicular to uTN
    uAB = np.zeros((1,2),dtype = "float32") 
    uAB[0][0] = uTN[1]
    uAB[0][1] = -uTN[0]
    
    # calculate four corners of the to-be-cropped portion of the image  
    #   use centroid to center the mouse
    A = N + uAB * (wid/2) + uTN * (length/2)
    B = N - uAB * (wid/2) + uTN * (length/2)
    C = N - uAB * (wid/2) - uTN * (length/2)
    D = N + uAB * (wid/2) - uTN * (length/2)
    
    # concatenate four corners into a np.array
    pts = np.concatenate((A,B,C,D))
    pts = pts.astype('float32')
    
    # generate the corresponding four corners in the cropped image
    dst = np.float32([[0,0],[wid,0],[wid,length],[0,length]])
    
    # generate transform matrix
    M = cv2.getPerspectiveTransform(pts,dst)

    # rotate and crop image
    warped = cv2.warpPerspective(image,M,(wid,length))
    
    return warped

# load the body_pose video 
# note that this step will need 32GiB of RAM for a 30min recording
body_video = skvideo.io.vread('A1_body.avi')[:,:,:,0]
frame_num = body_video.shape[0]
frame_ht = body_video.shape[1]
frame_wd = body_video.shape[2]

# load the DLC tracking of tailbase and centroid
tailbase_coords = label['tailbase'][['x','y']].values
centroid_coords = label['centroid'][['x','y']].values

# smoothening the tracking of tailbase and centorid with 1d gaussian filter
tailbase_coords_smooth = np.zeros_like(tailbase_coords)
centroid_coords_smooth = np.zeros_like(centroid_coords)

sig = 3

tailbase_coords_smooth[:,0] = gaussian_filter1d(tailbase_coords[:,0], sig, mode='nearest')
tailbase_coords_smooth[:,1] = gaussian_filter1d(tailbase_coords[:,1], sig, mode='nearest')

centroid_coords_smooth[:,0] = gaussian_filter1d(centroid_coords[:,0], sig, mode='nearest')
centroid_coords_smooth[:,1] = gaussian_filter1d(centroid_coords[:,1], sig, mode='nearest')

# center and align the body video
for i in tqdm(range(frame_num)):
    body_video[i] = four_point_transform(body_video[i], 
                                    tailbase_coords_smooth[i,0], 
                                    tailbase_coords_smooth[i,1], 
                                    centroid_coords_smooth[i,0], 
                                    centroid_coords_smooth[i,1], 
                                    frame_ht, 
                                    frame_wd)

# save the processed body video 
writer = skvideo.io.FFmpegWriter('centered_aligned_body.avi', outputdict={
            '-vcodec': 'mjpeg','-qscale': '1', '-b': '300000000', '-pix_fmt': 'yuv420p','-r':'25' })

for i in tqdm(range(np.asarray(body_video).shape[0])):
    writer.writeFrame(body_video[i])
writer.close()

# release space from RAM after the center_align_video has been saved to hard drive
del body_video
gc.collect()

100%|███████████████████████████████████| 45000/45000 [00:13<00:00, 3338.02it/s]


0

In [7]:
# calculate paw luminance, average paw luminance ratio, and paw luminance log-ratio

def cal_paw_luminance(label, ftir_video, size = 22):
    '''
    helper function for extracting the paw luminance signals of both hind paws from the ftir video
    
    input: 
    label: DLC tracking of the recording
    ftir_video: ftir video of the recording
    size: size of the cropping window centered on a paw
    output:
    hind_left: paw luminance of the left hind paw
    hind_right: paw luminance of the right hind paw
    '''
    
    num_of_frames = ftir_video.shape[0]
    # print(f'video length is {num_of_frames/25/60} mins')
        
    # right hind paw
    hind_right = []
    for i in tqdm(range(num_of_frames)):
        x,y = (int(label['hrpaw'][['x']].values[i]),int(label['hrpaw'][['y']].values[i]))
        hind_right.append(np.mean(ftir_video[i][y-size:y+size,x-size:x+size]))

    # left hind paw
    hind_left = []
    for i in tqdm(range(num_of_frames)):
        x,y = (int(label['hlpaw'][['x']].values[i]),int(label['hlpaw'][['y']].values[i]))
        hind_left.append(np.mean(ftir_video[i][y-size:y+size,x-size:x+size]))
        
    del ftir_video
    gc.collect()
    
    return hind_left, hind_right

# read ftir video
ftir_video = skvideo.io.vread('A1_ftir.avi')[:,:,:,0]

# calculate paw luminance
hind_left, hind_right = cal_paw_luminance(label, ftir_video, size = 22)

features['hind_left_luminance'] = hind_left
features['hind_right_luminance'] = hind_right

def scale_ftir(hind_left, hind_right):
    '''helper function for doing min 95-quntile scaler
       for individual recording, pool left paw and right paw ftir readings and find min and 95 percentile; then use those values to scale the readings'''
    
    
    left_paw = np.array(hind_left)
    right_paw = np.array(hind_right)
    
    min_ = min(np.nanmin(left_paw), np.nanmin(right_paw))
    max_ = max(np.nanmax(left_paw), np.nanmax(right_paw))
    quantile_ = np.nanquantile(np.concatenate([left_paw,right_paw]),.95)
    
    left_paw = (left_paw - min_) / (quantile_- min_)
    right_paw = (right_paw - min_) / (quantile_- min_)
    
    # replace all nan values with the mean, the nan values comes from DLC not tracking properly for those timepoints
    left_paw_mean = np.nanmean(left_paw)
    right_paw_mean = np.nanmean(right_paw)
    left_paw = np.nan_to_num(left_paw, nan = left_paw_mean)
    right_paw = np.nan_to_num(right_paw, nan = right_paw_mean)
    
    return (left_paw,right_paw)

hind_left_scaled, hind_right_scaled = scale_ftir(hind_left, hind_right)
features['hind_left_luminance_scaled'] = hind_left_scaled
features['hind_right_luminance_scaled'] = hind_right_scaled

# calculate luminance logratio
features['average_luminance_ratio'] = np.nansum(features['hind_left_luminance']) / np.nansum(features['hind_right_luminance']).reshape(-1,1)
features['luminance_logratio'] = np.log((features['hind_left_luminance_scaled']+1e-4)/(features['hind_right_luminance_scaled']+1e-4))

 72%|█████████████████████████▊          | 32331/45000 [00:37<00:14, 876.97it/s]/home/alexzhang/Documents/Woolf_lab/etho_pain/etho_pain_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/alexzhang/Documents/Woolf_lab/etho_pain/etho_pain_env/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|████████████████████████████████████| 45000/45000 [00:52<00:00, 864.61it/s]


In [8]:
# save extracted features 
with h5py.File('features.h5', 'w') as hdf:
    video = hdf.create_group('A1')
    for key in features.keys():
        video.create_dataset(key, data = features[key])

In [9]:
# load extracted features
features_load = h5py.File('features.h5', 'r')

In [10]:
features_load['A1'].keys()

<KeysViewHDF5 ['average_luminance_ratio', 'distance_traveled', 'hind_left_luminance', 'hind_left_luminance_scaled', 'hind_right_luminance', 'hind_right_luminance_scaled', 'luminance_logratio']>

In [11]:
features_load['A1']['luminance_logratio'][:]

array([-0.174779  , -0.1998102 , -0.11079095, ...,  0.39510472,
        0.35211906,  0.24669069])

In [12]:
features_load.close()